In [14]:
import hgvs.parser
import hgvs.dataproviders.uta
import hgvs.assemblymapper
import xml.etree.ElementTree as ET
import requests
import pandas as pd
from numpy import nan

# Take the table from Rafique et al., 2021 supplementary and annotate it 

## 1. Import and annotate the table from Rafique usinf Vcfanno 
the table was slightly cleaned and converted from MS Word to .csv

In [9]:
# MODY table
mody_df = pd.read_csv('input/clean_variants_from_ Rafique.csv',
                      converters={i: str for i in range(11)}, low_memory=False)
mody_df

,Gene,Nucleotide position,Protein position,Accession number,Gnom AD frequency,Country,Publication Year,Reference,ACMG\n (Intervar)*,transcript_stable_id,ref_protein_seq
0,GCK,c.908G>T,p.Arg303Leu,NM_001354800.1\n NP_001341729.1\n (25921421),,Greece,2015,-1,LP,,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...
1,GCK,c.748C>T,p.Arg250Cys,NP_000153.1\n (17204055),,Serbia,2006,-2,VUS,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...
2,GCK,c.182G>A,p.T61I,NP_000153.1\n (8433729),,Spain,2000,-3,VUS,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...
3,GCK,c.358C>T,p.A120T,NaN,,Spain,,-3,VUS,,
4,GCK,c.238delT,M238fsdelT,NP_000153.1,,Spain,,-3,NaN,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...
...,...,...,...,...,...,...,...,...,...,...,...
1012,HNF1A,c.1512C>A,p.Ser504Arg,NM_000545.6\n NP_000536.5,7.97E-06,China,2020,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...
1013,HNF1A,c.956-1G>C,,NM_000545.6\n NP_000536.5,,,,-238,,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...
1014,HNF1A,c. 347C>T,p.Ala116Val,NM_000545.6\n NP_000536.5\n (11315828),3.98E-06,,,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...
1015,HNF1A,c.1192C>G,p.Gln398Glu,NM_000545.6\n NP_000536.5,,,,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...


In [10]:
# initialize HGVS package
hp = hgvs.parser.Parser()
hdp = hgvs.dataproviders.uta.connect()
am = hgvs.assemblymapper.AssemblyMapper(hdp, assembly_name='GRCh38', alt_aln_method='splign', replace_reference=True)

In [11]:
# get the NM id based on an NP ID
def query_nm_id(db_name, NP_id):
    response = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=" + db_name + "&term=" + NP_id)
    root = ET.fromstring(response.content)

    NM_id = ''

    for child in root.iter('Id'):
        id = child.text

        full_response = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=' + db_name + '&id=' + id)

        if ('accession "NM_' in full_response.text):
            NM_id = 'NM_' + full_response.text.split('accession \"NM_', 1)[1].split('\"', 1)[0]

    return NM_id

In [12]:
NP_NM_map = {}	# Mapping of NP to NM accessions

# function to extract the NM accession from the 'Accession number' field, if present
def get_id_nm(acc_row):
    NP_id = ''
    NM_id = ''

    if ('NM_' in str(acc_row)):
        NM_id = 'NM_' + acc_row.split('NM_', 1)[1].split()[0]

    if ('NP_' in str(acc_row)):
        NP_id = 'NP_' + acc_row.split('NP_', 1)[1].split()[0]

        if (NM_id == ''):
            if (NP_id in NP_NM_map):
                NM_id = NP_NM_map[NP_id]
            else:
                NM_id = query_nm_id("protein", NP_id)
                NP_NM_map[NP_id] = NM_id
        else:
            NP_NM_map[NP_id] = NM_id

    return NM_id

In [15]:
mody_df['NM_acc'] = mody_df['Accession number'].apply(get_id_nm)

In [16]:
# get the genomic coordinates of a the variant
def get_genomic_coords(row):
    nm_acc = row['NM_acc']
    c_pos = row['Nucleotide position'].strip()
    c_pos = c_pos.replace(',', '')
    c_pos = c_pos.replace(' ', '')

    if ('NM' not in nm_acc) or ('c.' not in c_pos):
        return None

    hgvs_c = nm_acc + ':' + c_pos
    var_g = None

    try:
        var_c = hp.parse_hgvs_variant(hgvs_c)
        var_g = am.c_to_g(var_c)

    except:
        return None

    finally:
        return var_g

In [17]:
mody_df['DNA_coords'] = mody_df.apply(get_genomic_coords, axis=1)

Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000012.12&rettype=fasta&seq_start=120996276&seq_stop=120996296&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds


In [18]:
# Ensembl Exon variants table filtered and with all the alleles as separate rows and 'vf_allele' column added
Exon_var = pd.read_csv(
    'Ens_filtered_all_alleles_location_coord_no_duplicates.csv',
                      converters={i: str for i in range(11)}, low_memory=False)
Exon_var

,id,seq_region_name,start,end,strand,vf_allele,Location,coordinate,Gene,Transcript,Exon
0,rs1196762591,3,57227885.0,57227885.0,1.0,A,3:57227885,3:57227885:T>A,ENSG00000157500,ENST00000650354,ENSE00003837686
1,rs752732231,3,57227888.0,57227888.0,1.0,A,3:57227888,3:57227888:C>A,ENSG00000157500,ENST00000650354,ENSE00003837686
2,rs1559504574,3,57227889.0,57227892.0,1.0,GGG,3:57227889,3:57227889:GGGG>GGG,ENSG00000157500,ENST00000650354,ENSE00003837686
3,rs1022835366,3,57227890.0,57227890.0,1.0,A,3:57227890,3:57227890:G>A,ENSG00000157500,ENST00000650354,ENSE00003837686
4,rs1246196168,3,57227891.0,57227891.0,1.0,A,3:57227891,3:57227891:G>A,ENSG00000157500,ENST00000650354,ENSE00003837686
...,...,...,...,...,...,...,...,...,...,...,...
236708,rs759356936,10,100749850.0,100749850.0,1.0,T,10:100749850,10:100749850:C>T,ENSG00000075891,ENST00000554172,ENSE00002433876
236709,rs758091898,10,100781255.0,100781255.0,1.0,T,10:100781255,10:100781255:C>T,ENSG00000075891,ENST00000554172,ENSE00002435858
236710,rs200741999,10,100781275.0,100781275.0,1.0,G,10:100781275,10:100781275:A>G,ENSG00000075891,ENST00000554172,ENSE00002435858
236711,rs201925042,10,100781276.0,100781276.0,1.0,T,10:100781276,10:100781276:G>T,ENSG00000075891,ENST00000554172,ENSE00002435858


In [19]:
# map the row in the MODY table to the ensembl table, if possible
def map_to_ensembl(row):
    var_g = row['DNA_coords']
    if (var_g is None):
        return ''

    bp_pos = 0
    alt_allele = ''

    try:
        bp_pos = var_g.posedit.pos.start.base
        alt_allele = var_g.posedit.edit.alt

    except:
        return ''

    finally:
        # find corresponding position in the ensembl table
        ensembl_candidates = Exon_var[Exon_var['Location'].str.contains(str(bp_pos)) & (Exon_var['vf_allele'] == alt_allele)]

        if (len(ensembl_candidates) > 0):
            return ";".join(ensembl_candidates['id'].unique().tolist())
        else:
            return ''

mody_df['ensembl_id'] = mody_df.apply(map_to_ensembl, axis=1)
mody_df['DNA_coords'] = mody_df['DNA_coords'].apply(lambda x: str(x))

In [20]:
mody_df

,Gene,Nucleotide position,Protein position,Accession number,Gnom AD frequency,Country,Publication Year,Reference,ACMG\n (Intervar)*,transcript_stable_id,ref_protein_seq,NM_acc,DNA_coords,ensembl_id
0,GCK,c.908G>T,p.Arg303Leu,NM_001354800.1\n NP_001341729.1\n (25921421),,Greece,2015,-1,LP,,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_001354800.1,NC_000007.14:g.44146574C>A,rs1312678560
1,GCK,c.748C>T,p.Arg250Cys,NP_000153.1\n (17204055),,Serbia,2006,-2,VUS,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_000162,None,
2,GCK,c.182G>A,p.T61I,NP_000153.1\n (8433729),,Spain,2000,-3,VUS,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_000162,None,
3,GCK,c.358C>T,p.A120T,NaN,,Spain,,-3,VUS,,,,None,
4,GCK,c.238delT,M238fsdelT,NP_000153.1,,Spain,,-3,NaN,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_000162,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,HNF1A,c.1512C>A,p.Ser504Arg,NM_000545.6\n NP_000536.5,7.97E-06,China,2020,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120999278C>A,rs944413465
1013,HNF1A,c.956-1G>C,,NM_000545.6\n NP_000536.5,,,,-238,,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120996261G>C,
1014,HNF1A,c. 347C>T,p.Ala116Val,NM_000545.6\n NP_000536.5\n (11315828),3.98E-06,,,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120988853C>T,rs752886203
1015,HNF1A,c.1192C>G,p.Gln398Glu,NM_000545.6\n NP_000536.5,,,,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120996625C>G,


In [21]:
# Writing the annotated table to file
mody_df.to_csv(
    'Rafique_with_rs.csv',
    header=True, index=False)

In [22]:
#mody_df = pd.read_csv('/Users/ksenia/Documents/MODY_genes/whole_pipeline_311022/Rafique_with_rs.csv')
#mody_df = mody_df.replace(nan, '')
mody_df

,Gene,Nucleotide position,Protein position,Accession number,Gnom AD frequency,Country,Publication Year,Reference,ACMG\n (Intervar)*,transcript_stable_id,ref_protein_seq,NM_acc,DNA_coords,ensembl_id
0,GCK,c.908G>T,p.Arg303Leu,NM_001354800.1\n NP_001341729.1\n (25921421),,Greece,2015,-1,LP,,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_001354800.1,NC_000007.14:g.44146574C>A,rs1312678560
1,GCK,c.748C>T,p.Arg250Cys,NP_000153.1\n (17204055),,Serbia,2006,-2,VUS,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_000162,None,
2,GCK,c.182G>A,p.T61I,NP_000153.1\n (8433729),,Spain,2000,-3,VUS,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_000162,None,
3,GCK,c.358C>T,p.A120T,NaN,,Spain,,-3,VUS,,,,None,
4,GCK,c.238delT,M238fsdelT,NP_000153.1,,Spain,,-3,NaN,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_000162,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,HNF1A,c.1512C>A,p.Ser504Arg,NM_000545.6\n NP_000536.5,7.97E-06,China,2020,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120999278C>A,rs944413465
1013,HNF1A,c.956-1G>C,,NM_000545.6\n NP_000536.5,,,,-238,,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120996261G>C,
1014,HNF1A,c. 347C>T,p.Ala116Val,NM_000545.6\n NP_000536.5\n (11315828),3.98E-06,,,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120988853C>T,rs752886203
1015,HNF1A,c.1192C>G,p.Gln398Glu,NM_000545.6\n NP_000536.5,,,,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120996625C>G,


In [23]:
# Creating a list of variants from Rafique that now have the rs identifiers
ids_from_Rafique = []
for number in mody_df['ensembl_id']:
    if number != '': ids_from_Rafique.append(number)
len(ids_from_Rafique)

205

In [24]:
# Filtering the Ensembl table to only those variants
mapped_variants = Exon_var.drop_duplicates().query('id in @ids_from_Rafique').reset_index(drop=True)
mapped_variants

,id,seq_region_name,start,end,strand,vf_allele,Location,coordinate,Gene,Transcript,Exon
0,rs796065047,3,57238111.0,57238111.0,1.0,A,3:57238111,3:57238111:G>A,ENSG00000157500,ENST00000650354,ENSE00003522948
1,rs869320673,3,57260016.0,57260016.0,1.0,A,3:57260016,3:57260016:T>A,ENSG00000157500,ENST00000650354,ENSE00003516737
2,rs796065047,3,57238111.0,57238111.0,1.0,A,3:57238111,3:57238111:G>A,ENSG00000157500,ENST00000482800,ENSE00003527210
3,rs869320673,3,57260016.0,57260016.0,1.0,A,3:57260016,3:57260016:T>A,ENSG00000157500,ENST00000482800,ENSE00003585026
4,rs796065047,3,57238111.0,57238111.0,1.0,A,3:57238111,3:57238111:G>A,ENSG00000157500,ENST00000468342,ENSE00001849559
...,...,...,...,...,...,...,...,...,...,...,...
844,rs1379321773,2,10047653.0,10047653.0,1.0,G,2:10047653,2:10047653:A>G,ENSG00000172059,ENST00000401510,ENSE00001561024
845,rs1391231205,2,10043730.0,10043730.0,1.0,T,2:10043730,2:10043730:A>T,ENSG00000172059,ENST00000305883,ENSE00001265488
846,rs1379321773,2,10047653.0,10047653.0,1.0,G,2:10047653,2:10047653:A>G,ENSG00000172059,ENST00000305883,ENSE00001171571
847,rs1379321773,2,10047653.0,10047653.0,1.0,G,2:10047653,2:10047653:A>G,ENSG00000172059,ENST00000448523,ENSE00001701589


In [30]:
mapped_variants.to_csv('Rafique_mapped_to_Ens_1st.csv',
    header=True, index=False)

## 2. Table with Rafique annotated variants, but excluded BLK, KLF11 and PAX4

In [25]:
genes = ['GCK', 'HNF1A', 'HNF4A', 'HNF1B', 'INS', 'ABCC8', 'PDX1',
       'NEUROD1', 'KCNJ11', 'APPL1', 'CEL']

In [26]:
Rafique_excluded = mody_df.query('Gene in @genes').reset_index(drop=True)
Rafique_excluded = Rafique_excluded.replace(nan, '')
Rafique_excluded

,Gene,Nucleotide position,Protein position,Accession number,Gnom AD frequency,Country,Publication Year,Reference,ACMG\n (Intervar)*,transcript_stable_id,ref_protein_seq,NM_acc,DNA_coords,ensembl_id
0,GCK,c.908G>T,p.Arg303Leu,NM_001354800.1\n NP_001341729.1\n (25921421),,Greece,2015,-1,LP,,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_001354800.1,NC_000007.14:g.44146574C>A,rs1312678560
1,GCK,c.748C>T,p.Arg250Cys,NP_000153.1\n (17204055),,Serbia,2006,-2,VUS,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_000162,None,
2,GCK,c.182G>A,p.T61I,NP_000153.1\n (8433729),,Spain,2000,-3,VUS,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_000162,None,
3,GCK,c.358C>T,p.A120T,NaN,,Spain,,-3,VUS,,,,None,
4,GCK,c.238delT,M238fsdelT,NP_000153.1,,Spain,,-3,NaN,ENST00000403799.8,MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLR...,NM_000162,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,HNF1A,c.1512C>A,p.Ser504Arg,NM_000545.6\n NP_000536.5,7.97E-06,China,2020,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120999278C>A,rs944413465
988,HNF1A,c.956-1G>C,,NM_000545.6\n NP_000536.5,,,,-238,,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120996261G>C,
989,HNF1A,c. 347C>T,p.Ala116Val,NM_000545.6\n NP_000536.5\n (11315828),3.98E-06,,,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120988853C>T,rs752886203
990,HNF1A,c.1192C>G,p.Gln398Glu,NM_000545.6\n NP_000536.5,,,,-238,VUS,,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,NM_000545.6,NC_000012.12:g.120996625C>G,


In [27]:
# Creating a list of variants from Rafique that now have the rs identifiers
ids_from_Rafique_excluded = []
for number in Rafique_excluded['ensembl_id']:
    if number != '': ids_from_Rafique_excluded.append(number)
len(ids_from_Rafique_excluded)

195

In [28]:
mapped_variants_excluded = Exon_var.drop_duplicates().query('id in @ids_from_Rafique_excluded').reset_index(drop=True)
mapped_variants_excluded

,id,seq_region_name,start,end,strand,vf_allele,Location,coordinate,Gene,Transcript,Exon
0,rs796065047,3,57238111.0,57238111.0,1.0,A,3:57238111,3:57238111:G>A,ENSG00000157500,ENST00000650354,ENSE00003522948
1,rs869320673,3,57260016.0,57260016.0,1.0,A,3:57260016,3:57260016:T>A,ENSG00000157500,ENST00000650354,ENSE00003516737
2,rs796065047,3,57238111.0,57238111.0,1.0,A,3:57238111,3:57238111:G>A,ENSG00000157500,ENST00000482800,ENSE00003527210
3,rs869320673,3,57260016.0,57260016.0,1.0,A,3:57260016,3:57260016:T>A,ENSG00000157500,ENST00000482800,ENSE00003585026
4,rs796065047,3,57238111.0,57238111.0,1.0,A,3:57238111,3:57238111:G>A,ENSG00000157500,ENST00000468342,ENSE00001849559
...,...,...,...,...,...,...,...,...,...,...,...
806,rs193922400,11,17404524.0,17404524.0,1.0,T,11:17404524,11:17404524:C>T,ENSG00000006071,ENST00000643562,ENSE00003514460
807,rs193922400,11,17404524.0,17404524.0,1.0,T,11:17404524,11:17404524:C>T,ENSG00000006071,ENST00000528374,ENSE00002169001
808,rs72559734,11,17474955.0,17474955.0,1.0,T,11:17474955,11:17474955:C>T,ENSG00000006071,ENST00000683693,ENSE00003569766
809,rs193922400,11,17404524.0,17404524.0,1.0,T,11:17404524,11:17404524:C>T,ENSG00000006071,ENST00000531137,ENSE00002169689


In [29]:
# Writing to file specifying that it is the 1st stage of annotation ans short version of the table
mapped_variants.to_csv(
    'Rafique_mapped_to_Ens_1st_excluded_BLK_KLF11_PAX4.csv',
    header=True, index=False)